In [4]:
import torch , numpy as np
from torch.utils import data
import random
import matplotlib.pyplot as plt

In [3]:
#生成个数据集
def synthetic_data(w,b, num_examples):
    "生成y=wx+b+噪声"
    X = torch.normal(0,1,(num_examples, len(w)))
    y = torch.matmul(X, w)+b
    y +=torch.normal(0,0.01,y.shape)
    return X, y.reshape(-1, 1)

true_w = torch.tensor(([2, -3.4]))
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000)
features.shape, labels.shape

(torch.Size([1000, 2]), torch.Size([1000, 1]))

In [5]:
#读取数据集
def load_array(data_arrays, batch_size, is_train =True):
    "构造一个pytorch迭代器"
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, is_train)

batch_size =10
data_iter = load_array((features, labels), batch_size)

In [8]:
for step ,(x,y) in enumerate(data_iter):
    print(x, y)
    break

tensor([[ 0.6919,  0.6680],
        [ 0.5902,  1.4949],
        [-0.8719, -0.0312],
        [ 0.9119,  1.6818],
        [-0.7444,  1.1028],
        [-1.6454, -0.8870],
        [-0.0134,  0.5195],
        [ 0.3800, -0.5076],
        [-0.5317, -0.5010],
        [ 0.0722,  1.2702]]) False


In [28]:
#查看第一批次数据,iter生成迭代器，next拿第一个
print(next(iter(data_iter)))

[tensor([[ 1.5884, -1.0816],
        [ 0.0987, -1.9068],
        [ 1.0551, -0.3409],
        [ 0.1337,  2.1166],
        [-1.9544,  0.9358],
        [-0.6440, -1.5026],
        [-0.1950,  0.2906],
        [ 0.3930,  0.2760],
        [ 0.3485,  1.2801],
        [ 1.3077,  0.0649]]), tensor([[11.0398],
        [10.8923],
        [ 7.4673],
        [-2.7340],
        [-2.8858],
        [ 8.0121],
        [ 2.8488],
        [ 4.0453],
        [ 0.5485],
        [ 6.5857]])]


In [33]:
#定义模型
import torch.nn as nn

net = nn.Sequential(nn.Linear(2,1))

In [38]:
#c初始化模型参数
net[0].weight.data.normal_(0,0.01)
net[0].bias.data.fill_(0)

tensor([0.])

In [39]:
#损失函数 , 默认情况下，返回所有样本损失的平均值
loss = nn.MSELoss()

In [41]:
#定义优化器
trainer = torch.optim.SGD(net.parameters(), lr = 0.03)

In [45]:
#训练
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X), y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000098
epoch 2, loss 0.000098
epoch 3, loss 0.000097


In [51]:
w = net[0].weight.data
print('w的误差 ',true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的误差 ',true_b - b)

w的误差  tensor([-0.0001, -0.0002])
b的误差  tensor([-0.0002])
